In [1]:
import torch
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
import yaml
import os
import supervision as sv
import shutil
from ultralytics import YOLO
import yaml
import pandas as pd
import random
from collections import defaultdict
import random

Ultralytics YOLOv8.0.117  Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)
Setup complete  (12 CPUs, 31.9 GB RAM, 611.7/930.9 GB disk)


In [2]:
# file_path 경로에 _gether폴더 생성
file_path = "./datasets/OOA_Data_v13/"
_gether  = "gether/"
_test = "test/"
_train = "train/"
_valid = "valid/"
_test2 = "test2/"
_train2 = "train2/"
_valid2 = "valid2/"
_images = "images/"
_labels = "labels/"

gether_path = os.path.join(file_path, _gether)
gether_img_path = os.path.join(gether_path, _images)
gether_label_path = os.path.join(gether_path, _labels)

target_test_folder = os.path.join(file_path, _test2)
target_train_folder = os.path.join(file_path, _train2)
target_valid_folder = os.path.join(file_path, _valid2)

target_test_img_folder = os.path.join(target_test_folder, _images)
target_train_img_folder = os.path.join(target_train_folder, _images)
target_valid_img_folder = os.path.join(target_valid_folder, _images)

target_test_label_folder = os.path.join(target_test_folder, _labels)
target_train_label_folder = os.path.join(target_train_folder, _labels)
target_valid_label_folder = os.path.join(target_valid_folder, _labels)

random_seed = 42

In [5]:
def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print( folder_path + " : CREATED!")
    else:
        print( folder_path + " : EXISTS!")

def create_folders():
    create_folder(gether_path)
    create_folder(gether_img_path)
    create_folder(gether_label_path)
    
    create_folder(target_test_folder)
    create_folder(target_train_folder)
    create_folder(target_valid_folder)
    
    create_folder(target_test_img_folder)
    create_folder(target_train_img_folder)
    create_folder(target_valid_img_folder)
    
    create_folder(target_test_label_folder)
    create_folder(target_train_label_folder)
    create_folder(target_valid_label_folder)
    
    

# 폴더 통합 
def gether_to_onefolder(file_path):
    folders = []
    folders.append(os.path.join(file_path, _test))
    folders.append(os.path.join(file_path, _train))
    folders.append(os.path.join(file_path, _valid))
    
    # 이미지, 텍스트 복사 이동
    for folder in folders:
        img_folder = os.path.join(folder, _images)
        img_files = os.listdir(img_folder)
        for img_file in img_files:
            img_src = os.path.join(img_folder, img_file)
            img_dst = os.path.join(gether_img_path, img_file)
            # img_src 폴더에서 img_dst폴더로
            if not os.path.exists(img_dst):
                shutil.copy(img_src, img_dst)
            
        lab_folder = os.path.join(folder, _labels)
        lab_files = os.listdir(lab_folder)
        for lab_file in lab_files:
            lab_src = os.path.join(lab_folder, lab_file)
            lab_dst = os.path.join(gether_label_path, lab_file)
            # img_src 폴더에서 img_dst폴더로
            if not os.path.exists(lab_dst):
                shutil.copy(lab_src, lab_dst)
                
# yaml 파일에서 names 반환
def get_yaml_names(file_path):
    # YAML 파일 열기
    with open(file_path, 'r') as file:
        # YAML 데이터 로드
        data = yaml.safe_load(file)

    # names 값 가져오기
    return data['names']

# 전체 class별 숫자 반환
def Get_numbers_count(names, gether_label_path):
    numbers_count = {}  # 숫자별 개수를 저장할 딕셔너리 초기화

    # gether_img_path 경로의 텍스트 파일들 순회
    for filename in os.listdir(gether_label_path):
        _path = os.path.join(gether_label_path, filename)
        
        # 텍스트 파일 열기
        with open(_path, 'r') as file:
            lines = file.readlines()
        
        # 각 줄에서 첫 번째 숫자 추출하여 개수 세기
        for line in lines:
            number = line.split()[0]
            numbers_count[number] = numbers_count.get(number, 0) + 1
    
    df = pd.DataFrame(numbers_count.items(), columns=['Class_id', 'Count'])
    
    # 열을 숫자형으로 변환
    df['Class_id'] = df['Class_id'].astype(int)
    df['Count'] = df['Count'].astype(int)

    # Name 컬럼 생성 ( Class_id로 부터 )
    df['Name'] = [names[number] for number in df['Class_id']]

    # Class_id 정렬
    df = df.sort_values('Class_id', ascending=True)
    
    return df

# input 
#   gether_label_path : 라벨을 모으고 싶은 폴더 경로
# return 
#   pd.dataframe
# 파일별로 포함된 class_id 반환
def get_numbers_each_file(gether_label_path):
    data = []

    # gether_label_path 폴더 내의 텍스트 파일들 순회
    for filename in os.listdir(gether_label_path):
        file_path = os.path.join(gether_label_path, filename)
        
        # 파일명
        file_name = filename
        
        # 파일 내용
        with open(file_path, 'r') as file:
            lines = file.readlines()
        
        # 파일 내용에서 Number 추출
        numbers = []
        for line in lines:
            number = int(line.split()[0])
            numbers.append(number)
        
        # 데이터 추가
        data.append({'file_name': file_name, 'Numbers': numbers})

    # 데이터프레임 생성
    df = pd.DataFrame(data)
    return df

# 
def divide_txt_files(source_folder, target_folder_1, target_folder_2, target_folder_3, ratio_1, ratio_2, ratio_3, random_seed=42):
    assert ratio_1 + ratio_2 + ratio_3 == 1, "비율 합계는 1이어야 합니다."

    if random_seed is not None:
        random.seed(random_seed)

    # 대상 폴더 생성
    os.makedirs(target_folder_1, exist_ok=True)
    os.makedirs(target_folder_2, exist_ok=True)
    os.makedirs(target_folder_3, exist_ok=True)

    # 폴더별 번호별 파일 개수 초기화

    df = get_numbers_each_file(gether_label_path)
    # 데이터 프레임 섞기
    df = df.sample(frac=1, random_state=random_seed)

    # 각 행을 반복
    for _, row in df.iterrows():
        # 파일데이터 가져오기
        file_name, numbers = row["file_name"], row["Numbers"]
        source_file_path = os.path.join(source_folder, file_name)

        # 원본 파일 체크
        if not os.path.isfile(source_file_path):
            print(f" {file_name} 파일이 없습니다.")
            continue

        # 현재 개수에 따라 분배
        for number in sorted(numbers):
            current_counts = counts[number]
            total = sum(current_counts)
            if total == 0 or (current_counts[0] / total < ratio_1):
                target_folder = target_folder_1
                counts[number][0] += 1
            elif current_counts[1] / total < ratio_2:
                target_folder = target_folder_2
                counts[number][1] += 1
            else:
                target_folder = target_folder_3
                counts[number][2] += 1

            # 대상폴더로 복제
            target_file_path = os.path.join(target_folder, file_name)
            shutil.copy(source_file_path, target_file_path)

            break
        
def divide_img_files(source_folder, label_folder, target_folder):
    # 폴더 체크
    os.makedirs(target_folder, exist_ok=True)

    # 파일이름 가져오기 
    label_files = [os.path.splitext(file)[0] for file in os.listdir(label_folder)]

    for file_name in os.listdir(source_folder):
        # 확장자 제외
        file_name_without_ext = os.path.splitext(file_name)[0]

        # 같은이름 파일 옮기기
        if file_name_without_ext in label_files:
            source_file_path = os.path.join(source_folder, file_name)
            target_file_path = os.path.join(target_folder, file_name)
            shutil.move(source_file_path, target_file_path)

In [103]:
create_folders()
gether_to_onefolder(file_path)
# names 생성
names = get_yaml_names(os.path.join(file_path, 'data.yaml'))

./datasets/OOA_Data_v13/gether/ Folder already exists.
./datasets/OOA_Data_v13/gether/images/ Folder already exists.
./datasets/OOA_Data_v13/gether/labels/ Folder already exists.
./datasets/OOA_Data_v13/test2/ Folder already exists.
./datasets/OOA_Data_v13/train2/ Folder already exists.
./datasets/OOA_Data_v13/valid2/ Folder already exists.
./datasets/OOA_Data_v13/test2/images/ Folder already exists.
./datasets/OOA_Data_v13/train2/images/ Folder already exists.
./datasets/OOA_Data_v13/valid2/images/ Folder already exists.
./datasets/OOA_Data_v13/test2/labels/ Folder already exists.
./datasets/OOA_Data_v13/train2/labels/ Folder already exists.
./datasets/OOA_Data_v13/valid2/labels/ Folder already exists.


In [139]:
# 텍스트 분류 이동

# 분배 비율
ratio_1 = 0.8
ratio_2 = 0.1
ratio_3 = 0.1

divide_txt_files(gether_label_path,target_train_label_folder, target_test_label_folder, target_valid_label_folder,ratio_1, ratio_2, ratio_3)

In [4]:
# 이미지 분류 이동
# 텍스트와 같은 위치로 이동됨
divide_img_files(gether_img_path,target_test_label_folder, target_test_img_folder)
divide_img_files(gether_img_path,target_train_label_folder, target_train_img_folder)
divide_img_files(gether_img_path,target_valid_label_folder, target_valid_img_folder)

In [6]:

names = get_yaml_names(os.path.join(file_path, 'data.yaml'))

In [8]:
df_nums = Get_numbers_count(names=names, gether_label_path=target_train_label_folder)
df_nums

,Class_id,Count,Name
17,0,385,back_pack
18,1,272,bike
22,2,212,bike_helmet
21,3,258,bookcase
19,4,142,bottle
5,5,234,calculator
29,6,379,desk_chair
13,7,220,desk_lamp
6,8,168,desktop_computer
0,9,145,file_cabinet


In [ ]:
df_train_nums = Get_numbers_count(names=names, gether_label_path=target_train_label_folder)
df_train_nums.head()